Getting the station list from page: https://www.veturilo.waw.pl/mapa-stacji/ it is going to provide us with the station adress, 
geo co-ordinates and the amount of available bikes. We don't want to send someone to the station with 0 bikes. 

#### Getting the stations co-ordinates

In [1]:
#importing libraries required for web scraping 

import requests
import lxml.html as lh
import pandas as pd

In [2]:
#Get the whole page
url='https://www.veturilo.waw.pl/mapa-stacji/'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [13]:
#Get headers
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [14]:
col

[('Lokalizacja', []),
 ('Dostępne rowery', []),
 ('Ilość stojaków', []),
 ('Wolne stojaki', []),
 ('Współrzędne', [])]

In [21]:
#Get the table data
for j in range(1,len(tr_elements)):
    T=tr_elements[j] 
    if len(T)!=5:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [16]:
#check the lenght to make sure all of the columns have the same size
[len(C) for (title,C) in col]

[394, 394, 394, 394, 394]

In [43]:
#Assign header to the table
Dict={title:column for (title,column) in col}
Bike_Stations=pd.DataFrame(Dict)

#### Cleaning data

In [44]:
#Translate column names from polish to english
Bike_Stations.columns = ['Adress', 'Available_Bikes', 'Bike_Stands','Empty_Stands','Coordinates']

In [45]:
#Drop the columns we don't need Bike_Stands, Empty_Stands.
#Even if there are no emty stands you can leave the bike on the station

In [46]:
del Bike_Stations['Bike_Stands']
del Bike_Stations['Empty_Stands']

In [47]:
#Drop the rows that have 0 available bikes - as stated before we don't want to sent user to the station that has no bikes
Bike_Stations = Bike_Stations[Bike_Stations.Available_Bikes != 0]

In [49]:
#Splitting co-ordinates into 2 rows Latitude and Longtitude
Bike_Stations = Bike_Stations.join(Bike_Stations['Coordinates'].str.split(',', 1, expand=True).rename(columns={0:'Latitude', 1:'Longitude'}))
                                  

In [50]:
#drop Coordinates column
del Bike_Stations['Coordinates']

In [51]:
#check the table to see if we have desired output
Bike_Stations.head()

,Adress,Available_Bikes,Latitude,Longitude
0,1 Sierpnia - DK Włochy,14,52.1946888,20.9667823
1,11 Listopada - Ratuszowa,19,52.261449,21.037614
2,11 listopada - Środkowa,14,52.263109,21.0381895
3,Abrahama - Kapelanów AK,11,52.22837,21.08581
4,Afrykańska - Egipska,15,52.2251969,21.0712892
